This notebook explores a simple hypothesis test checking whether the accuracy of a trained model for binary classification is meaningfully different from a majority class baseline.  We test this making a parametric assumption: we assume that the binary correct/incorrect results follow a binomial distribution (and approximate the binomial with a normal distribution).

In [1]:
import sys
from collections import Counter
from sklearn import preprocessing
from sklearn import linear_model
import pandas as pd
from scipy import sparse
import numpy as np
from math import sqrt 
from scipy.stats import norm

In [2]:
def read_data(filename):
    X=[]
    Y=[]
    with open(filename, encoding="utf-8") as file:
        for line in file:
            cols=line.rstrip().split("\t")
            label=cols[0]
            text=cols[1]
            X.append(text)
            Y.append(label)
    return X, Y

In [17]:
# The directory should contain train.tsv, dev.tsv and test.tsv
directory="../data/lmrd"

In [18]:
trainX, trainY=read_data("%s/train.tsv" % directory)
devX, devY=read_data("%s/dev.tsv" % directory)

In [19]:
def majority_class(trainY, devY):
    labelCounts=Counter()
    for label in trainY:
        labelCounts[label]+=1
    majority=labelCounts.most_common(1)[0][0]
    
    correct=0.
    for label in devY:
        if label == majority:
            correct+=1
            
    print("%s\t%.3f" % (majority, correct/len(devY)))
    return correct/len(devY)

In [6]:
# Here's a sample dictionary we can create by inspecting the output of the Mann-Whitney test (in 2.compare/)
dem_dictionary=set(["republican","cut", "opposition"])
repub_dictionary=set(["growth","economy"])

def political_dictionary_feature(tokens):
    feats={}
    for word in tokens:
        if word in dem_dictionary:
            feats["word_in_dem_dictionary"]=1
        if word in repub_dictionary:
            feats["word_in_repub_dictionary"]=1
    return feats

In [20]:
# start with a fixed dict to look up keywords occuring in the review
pos_dictionary = set(["like", "love", "good"])
neg_dictionary = set(["dislike", "hate", "bad"])

def sentiment_dictionary_feature(tokens):
    feats={}
    for word in tokens:
        if word in pos_dictionary:
            feats["word_in_pos_dictionary"]=1
        if word in neg_dictionary:
            feats["word_in_neg_dictionary"]=1
    return feats

In [32]:
# add a keyword feature set
pos_cap_dict = set(["WOW", "ABSOLUTELY", "SO", "EVER"])
neg_cap_dict = set(["NOT", "VERY", "BUT", "?", "ONLY", "NO", "WTF", "...", "SPENT", "$"])

def new_feature_class_two(tokens):
    feats={}
    for word in tokens:
        if word in pos_cap_dict:
            feats["allcaps_in_pos_dict"]=1
        if word in neg_cap_dict:
            feats["allcaps_in_neg_dict"]=1
    return feats

In [21]:
def unigram_feature(tokens):
    feats={}
    for word in tokens:
        feats["UNIGRAM_%s" % word]=1
    return feats

In [22]:
def build_features(trainX, feature_functions):
    data=[]
    for doc in trainX:
        feats={}

        # sample text data is already tokenized; if yours is not, do so here
        tokens=doc.split(" ")
        
        for function in feature_functions:
            feats.update(function(tokens))

        data.append(feats)
    return data

In [23]:
# This helper function converts a dictionary of feature names to unique numerical ids
def create_vocab(data):
    feature_vocab={}
    idx=0
    for doc in data:
        for feat in doc:
            if feat not in feature_vocab:
                feature_vocab[feat]=idx
                idx+=1
                
    return feature_vocab

In [24]:
# This helper function converts a dictionary of feature names to a sparse representation
# that we can fit in a scikit-learn model.  This is important because almost all feature 
# values will be 0 for most documents (note: why?), and we don't want to save them all in 
# memory.

def features_to_ids(data, feature_vocab):
    new_data=sparse.lil_matrix((len(data), len(feature_vocab)))
    for idx,doc in enumerate(data):
        for f in doc:
            if f in feature_vocab:
                new_data[idx,feature_vocab[f]]=doc[f]
    return new_data

In [25]:
# This function evaluates a list of feature functions on the training/dev data arguments
def pipeline(trainX, devX, trainY, devY, feature_functions):
    trainX_feat=build_features(trainX, feature_functions)
    devX_feat=build_features(devX, feature_functions)

    # just create vocabulary from features in *training* data
    feature_vocab=create_vocab(trainX_feat)

    trainX_ids=features_to_ids(trainX_feat, feature_vocab)
    devX_ids=features_to_ids(devX_feat, feature_vocab)
    
    logreg = linear_model.LogisticRegression(C=1.0, solver='lbfgs', penalty='l2', max_iter=10000)
    logreg.fit(trainX_ids, trainY)
    print("Accuracy: %.3f" % logreg.score(devX_ids, devY))  

In [26]:
# This function trains a model and returns the predicted and true labels for test data
def evaluate(trainX, devX, trainY, devY, feature_functions):
    trainX_feat=build_features(trainX, feature_functions)
    devX_feat=build_features(devX, feature_functions)

    # just create vocabulary from features in *training* data
    feature_vocab=create_vocab(trainX_feat)

    trainX_ids=features_to_ids(trainX_feat, feature_vocab)
    devX_ids=features_to_ids(devX_feat, feature_vocab)
    
    logreg = linear_model.LogisticRegression(C=1.0, solver='lbfgs', penalty='l2', max_iter=10000)
    logreg.fit(trainX_ids, trainY)
    predictions=logreg.predict(devX_ids)
    return (predictions, devY)

In [27]:
baseline=majority_class(trainY,devY)

pos	0.500


In [28]:
def binomial_test(predictions, truth, baseline, significance_level=0.95):
    correct=[]
    for pred, gold in zip(predictions, truth):
        correct.append(int(pred==gold))
        
    success_rate=np.mean(correct)

    # two-tailed test
    critical_value=(1-significance_level)/2
    # ppf finds z such that p(X < z) = critical_value
    z_alpha=-1*norm.ppf(critical_value)
    print("Critical value: %.3f\tz_alpha: %.3f" % (critical_value, z_alpha))
    
    # the standard error is the square root of the variance/sample size
    # the variance for a binomial test is p*(1-p)
    standard_error=sqrt((success_rate*(1-success_rate))/len(correct))

    Z=(success_rate-baseline)/standard_error
    lower=success_rate-z_alpha*standard_error
    upper=success_rate+z_alpha*standard_error
    pval=norm.cdf(-abs(Z))
    print ("Accuracy: %.3f, n = %s" % (success_rate, len(correct)))
    print("%s%% Confidence interval: [%.3f,%.3f]" % (significance_level*100, lower, upper))

    print("Z score: %.3f" % Z)
    print("p-value: %.5f" % pval)

    print ("Critical region corresponding to z_alpha=[%.3f,%.3f]: [%.3f, %.3f]" % (-z_alpha, z_alpha, baseline-z_alpha*standard_error, baseline+z_alpha*standard_error))
    print ("Can we reject null that %.3f is different from %.3f at %s significance level? %s" % (success_rate, baseline, significance_level*100, "Yes" if Z < -z_alpha or Z > z_alpha else "No"))

In [15]:
features=[political_dictionary_feature]
predictions, truth=evaluate(trainX, devX, trainY, devY, features)
binomial_test(predictions, truth, baseline, significance_level=.95)

Critical value: 0.025	z_alpha: 1.960
Accuracy: 0.541, n = 257
95.0% Confidence interval: [0.480,0.602]
Z score: 1.127
p-value: 0.12996
Critical region corresponding to z_alpha=[-1.960,1.960]: [0.445, 0.567]
Can we reject null that 0.541 is different from 0.506 at 95.0 significance level? No


In [31]:
features=[sentiment_dictionary_feature]
predictions, truth=evaluate(trainX, devX, trainY, devY, features)
binomial_test(predictions, truth, baseline, significance_level=.95)

Critical value: 0.025	z_alpha: 1.960
Accuracy: 0.610, n = 5000
95.0% Confidence interval: [0.596,0.623]
Z score: 15.886
p-value: 0.00000
Critical region corresponding to z_alpha=[-1.960,1.960]: [0.486, 0.514]
Can we reject null that 0.610 is different from 0.500 at 95.0 significance level? Yes


In [33]:
features=[new_feature_class_two]
predictions, truth=evaluate(trainX, devX, trainY, devY, features)
binomial_test(predictions, truth, baseline, significance_level=.95)

Critical value: 0.025	z_alpha: 1.960
Accuracy: 0.598, n = 5000
95.0% Confidence interval: [0.584,0.611]
Z score: 14.073
p-value: 0.00000
Critical region corresponding to z_alpha=[-1.960,1.960]: [0.486, 0.514]
Can we reject null that 0.598 is different from 0.500 at 95.0 significance level? Yes


**TODO**: Now apply this same method to the models you just submitted for the last homework.  Are the features you created significantly better than a majority class baseline? Is one significantly better than the other?

